Use case: a company owns a job portal and has many job posts from other companies who wanna hire candidates. The company wants to have understanding of these job feeds in order to build a search engine. Your job is to do analysis on these data and feedback insights to the business

In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import re

In [2]:
url = 'https://drive.google.com/file/d/16EGubjXaeDwTH7sc6FnuQj7EI9ZfRYNL/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
data = pd.read_csv(path, sep = '\t')

### Perform proper analysis on the data
### Generate Word Cloud, you can follow this if you like https://www.datacamp.com/tutorial/wordcloud-python
### Apply data cleaning standard methods on the data to preprocess it
### Generate Word2Vec embeddings
### Try similarity functions of the word2vec
### Visiulaize some embeddings
### implement a streamlit and deploy your application

## Resources:
https://jalammar.github.io/illustrated-word2vec/

https://www.youtube.com/watch?v=hQwFeIupNP0

https://towardsdatascience.com/a-beginners-guide-to-word-embedding-with-gensim-word2vec-model-5970fa56cc92

https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook

In [3]:
data.columns

Index(['Unnamed: 0', 'job_title', 'full_description', 'company', 'location'], dtype='object')

In [4]:
data = data[['job_title', 'full_description', 'company', 'location']]

In [5]:
data

,job_title,full_description,company,location
0,Senior Testmanager/Qualitätsmanager (m/w) Shif...,"Zum weiteren Ausbau unseres Teams, suchen wir ...",NTT DATA Deutschland GmbH,deutschlandweit
1,Junior Java Entwickler (m/w) in spannenden Ent...,Erfahrungen im Umgang mit objektorientierten P...,NTT DATA Deutschland GmbH,"Frankfurt a.M., Köln, München, Stuttgart, deut..."
2,(Senior)Technical Consultant (m/w) Testautomat...,DBS-8346-SR - Einsatzort: deutschlandweit - Wi...,NTT DATA Deutschland GmbH,deutschlandweit
3,Einsteiger (w/m) Digitalisierung und Data Anal...,Die globale EY-Organisation* ist einer der Mar...,Ernst & Young GmbH Wirtschaftsprüfungsgesellsc...,Deutschlandweit
4,Projektmanager / Managing Consultant (m/w) Cus...,Idealerweise über drei Jahre praktische Erfahr...,NTT DATA Deutschland GmbH,"Frankfurt a.M., Köln, München, deutschlandweit"
...,...,...,...,...
23477,Technischer Fachberater (m/w) im Innendienst,Sie haben eine kaufmännische bzw. technische A...,Rala GmbH & Co. KG,Ludwigshafen
23478,Wirtschaftsingenieur als Referent im Vertrieb ...,Die Deutsche Bahn ist nicht nur einer der wich...,Deutsche Bahn AG,Berlin
23479,IT Fachkraft m/w,"Standort: Heilbad Heiligenstadt (37308), Thüri...",Magna,Heilbad Heiligenstadt
23480,(Junior) Projektingenieur Sensorik und Kommuni...,Für unseren Geschäftsbereich Verteidigung & Si...,IABG Industrieanlagen - Betriebsgesellschaft mbH,"Ottobrunn bei München, Koblenz"


In [6]:
data.isna().sum()

job_title           0
full_description    0
company             0
location            0
dtype: int64

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23482 entries, 0 to 23481
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   job_title         23482 non-null  object
 1   full_description  23482 non-null  object
 2   company           23482 non-null  object
 3   location          23482 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB


## Generate Word2Vec embeddings:



In [8]:
# before moving to Word2Vec , convert "job_title" column entries to lower case
data['job_title_lower'] = data['job_title'].str.lower()

In [9]:
# remove commas
data['job_title_lower'] = data['job_title_lower'].str.replace(',', '')

In [10]:
# remove (m/w)  and (w/m)
data['job_title_lower'] = data['job_title_lower'].str.replace(r'\s*\([mw][/][mw]\)', '', regex=True)


In [11]:
data["job_title_lower"].head()

0       senior testmanager/qualitätsmanager shift left
1    junior java entwickler in spannenden enterpris...
2     (senior)technical consultant testautomatisierung
3    einsteiger digitalisierung und data analytics ...
4    projektmanager / managing consultant customer ...
Name: job_title_lower, dtype: object

### Word2Vec(sentences, size, window, min_count) : the parameter "sentences"  requires that a format of ‘list of lists’ for training where every document(every entry in that column) contains list of words and every list(i,e every row or every entry )contains lists of tokens of that document

In [12]:
import gensim
from gensim.models import Word2Vec

In [13]:
#df['job_title_lower'].str.split(" ").tolist()
sentence_list = [row.split(",") for row in data["job_title_lower"]]

In [14]:
model_1 = Word2Vec(sentence_list, min_count =1 , vector_size = 50, workers=3, window =3, sg = 1)

## Access the Word Vectors

In [15]:
data["job_title_lower"][101]

'junior java-backend-entwickler'

In [16]:
data["job_title_lower"][1001]

'sap-entwickler im bereich handel'

In [17]:
model_1.wv["dms-entwickler"]

array([ 0.01536464,  0.00799956, -0.00539816,  0.01923706,  0.01517757,
       -0.0091335 , -0.00554757,  0.01485211,  0.01494465, -0.00796159,
        0.01563065, -0.00351606, -0.00640683,  0.01519188, -0.011118  ,
        0.00373998, -0.01438597, -0.01828506,  0.01188127, -0.01102711,
        0.01589881, -0.01316178, -0.00821611,  0.00126515, -0.00196037,
       -0.01107844, -0.0141155 , -0.00476068,  0.01442409, -0.00105693,
       -0.01195335, -0.01361152,  0.01185393,  0.00726946, -0.01293606,
        0.00860211,  0.00680897,  0.00153702,  0.01603754, -0.01222179,
        0.00789344,  0.00271348,  0.00575639, -0.00895321,  0.00388751,
       -0.00394876,  0.01965477, -0.00852337,  0.0065969 ,  0.0100542 ],
      dtype=float32)

## Compute similarities:

We also can use the built-in function model.wv.most_similar() to get a set of the most similar words for a given word based on the Euclidean distance.



In [18]:
model_1.wv.most_similar('dms-entwickler')

[('kfz-mechatroniker im versuchsbau', 0.5344768166542053),
 ('systemtester steuergeräte automotive', 0.5169220566749573),
 ('sap-anwendungsbetreuer mm/wm und qm', 0.5016372799873352),
 ('vergabemanager / procurement specialist planungs- beratungs- und baulösungen',
  0.4737108051776886),
 ('it business consultant vertrieb digital signage & elektronische preisauszeichnung',
  0.45194876194000244),
 ('informatiker als experte management it-architektur', 0.4449036419391632),
 ('abschlussarbeit entwicklung eines sgemented lcd-moduls als preiswerter ersatz für ein dot-matrix lcd als display für ein elektrisch angetriebenes motorrad',
  0.44434770941734314),
 ('it application manager e-commerce / fulfillment', 0.4442838728427887),
 ('database support engineer integration', 0.44139790534973145),
 ('gärtnermeister/in oder gartenbauingenieur/in', 0.4407973885536194)]

In [19]:
model_1.wv.similarity('dms-entwickler', 'junior java-backend-entwickler')

0.09598739

# Cosine Similarity:

In [20]:
def cosine_similarity (model, word, target_list_words, num) :
    cosine_dict ={}
    word_list = []
    a = model.wv[word]
    for item in target_list_words :
        if item != word :
            b = model.wv[item]
            cos_sim = np.dot(a, b)/(norm(a)*norm(b))
            cosine_dict[item] = cos_sim

  #'x' is the parameter of the lambda function. It represents each item (key-value pair) from the cosine_dict.items() list during the sorting process.
  # x[1] accesses the second element of the tuple (key-value pair), which corresponds to the value of the key-value pair.
    dist_sort=sorted(cosine_dict.items(), key=lambda x: x[1],reverse = True) ## in Descedning order

  #returns 'num' set of cosine-similarities between target_list_words and word
    for item in dist_sort:
        word_list.append((item[0], item[1]))
    return word_list[0:num]

In [21]:
# only get the unique Maker_Model
target_list_words = list(data['job_title_lower'].unique())

In [22]:
# call the cosine_similarity function
keyword = input("Enter a key word")
set_similar_jobs = cosine_similarity (model_1, keyword, target_list_words, 15)
set = [item[0] for item in set_similar_jobs]
set

Enter a key worddms-entwickler


['kfz-mechatroniker im versuchsbau',
 'systemtester steuergeräte automotive',
 'sap-anwendungsbetreuer mm/wm und qm',
 'vergabemanager / procurement specialist planungs- beratungs- und baulösungen',
 'it business consultant vertrieb digital signage & elektronische preisauszeichnung',
 'informatiker als experte management it-architektur',
 'abschlussarbeit entwicklung eines sgemented lcd-moduls als preiswerter ersatz für ein dot-matrix lcd als display für ein elektrisch angetriebenes motorrad',
 'it application manager e-commerce / fulfillment',
 'database support engineer integration',
 'gärtnermeister/in oder gartenbauingenieur/in',
 'softwareentwickler im robotikbereich',
 'service manager im innovations-team der corporate academy in stuttgart',
 'executive partner risikomanagement und controlling',
 'expert technical support automatisierungstechnik',
 'senior referent konzernrechnungswesen']

In [23]:
new_df= data.loc[data['job_title_lower'].isin(set), ["job_title", "full_description", "company", "location"]]
new_df

,job_title,full_description,company,location
934,Expert Technical Support (w/m) Automatisierung...,"Ottobrunn bei München, Germany - Werden Sie Te...",Panasonic Electric Works Europe AG,Ottobrunn bei München
1491,Executive Partner Risikomanagement und Control...,Gestaltung und Weiterentwicklung des Geschäfts...,msgGillardon AG,"Berlin, Frankfurt, München, Stuttgart"
4130,Softwareentwickler im Robotikbereich (m/w),Würzburg - Sie schätzen neue Herausforderungen...,Brunel GmbH,Würzburg
4432,Database Support Engineer (m/w) Integration,Jetzt Database Support Engineer (m/w) Integrat...,ING-DiBa AG,Nürnberg
5106,Abschlussarbeit Entwicklung eines Sgemented LC...,Ludwigsburg * Bachelor-/Master-/Diplom-Arbeite...,Robert Bosch GmbH,Ludwigsburg
7762,Gärtnermeister/in oder Gartenbauingenieur/in,eine/n Gärtnermeister/in oder Gartenbauingenie...,Bau & Service Oberursel (BSO),Oberursel (Taunus)
8942,Informatiker als Experte (m/w) Management IT-A...,"Bei den Stadtwerken München (SWM) wissen wir, ...",Stadtwerke München GmbH,München
9259,Kfz-Mechatroniker (w/m) im Versuchsbau,Für unsere Niederlassung in Böblingen suchen w...,Formel D Group,Böblingen
11994,IT Business Consultant Vertrieb (w/m) Digital ...,Beratung des Fachbereichs Vertrieb und Entwick...,Kaufland,Weinsberg
13918,Vergabemanager / Procurement Specialist (m/w) ...,Abgeschlossenes Studium als Bauingenieur (m/w)...,Drees & Sommer Frankfurt GmbH,Frankfurt am Main
